## <span style="color:darkgreen">IBM Watson AutoAI - Generated Scikit-learn Notebook v1.14.4</span>

Consider these tips for working with an auto-generated notebook:
- Notebook code generated using AutoAI will execute successfully. If you modify the notebook, we cannot guarantee it will run successfully.
- This pipeline is optimized for the original data set. The pipeline might fail or produce sub-optimum results if used with different data.  If you want to use a different data set, consider retraining the AutoAI experiment to generate a new pipeline. For more information, see <a href="https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/autoai-notebook.html">Cloud Platform</a> 
- Before modifying the pipeline or trying to re-fit the pipeline, consider that the code converts dataframes to numpy arrays before fitting the pipeline (a current restriction of the preprocessor pipeline).


<a id="content"></a>
## Notebook content

This notebook contains a Scikit-learn representation of AutoAI pipeline. This notebook introduces commands for getting data, training the model, and testing the model. 

Some familiarity with Python is helpful. This notebook uses Python 3.7 and scikit-learn 0.23.1.

## Notebook goals

-  Scikit-learn pipeline definition
-  Pipeline training 
-  Pipeline evaluation

## Contents

This notebook contains the following parts:

**[Setup](#setup)**      
&nbsp;&nbsp;[Package installation](#install)   
&nbsp;&nbsp;[AutoAI experiment metadata](#variables_definition)   
**[Pipeline inspection](#inspection)**      
&nbsp;&nbsp;[Read training data](#read)    
&nbsp;&nbsp;[Train and test data split](#split)    
&nbsp;&nbsp;[Make pipeline](#preview_model_to_python_code)      
&nbsp;&nbsp;[Train pipeline model](#train)  
&nbsp;&nbsp;[Test pipeline model](#test_model)  
**[Next steps](#next_steps)**   
**[Copyrights](#copyrights)**

<a id="setup"></a>
# Setup

<a id="install"></a>
## Package installation
Before you use the sample code in this notebook, install the following packages:
 - ibm_watson_machine_learning,
 - autoai-libs.


In [ ]:
!pip install ibm-watson-machine-learning | tail -n 1
!pip install -U autoai-libs | tail -n 1

<a id="variables_definition"></a>
## AutoAI experiment metadata
The following cell contains the training data connection details.  
**Note**: The connection might contain authorization credentials, so be careful when sharing the notebook.

In [ ]:
# @hidden_cell
from ibm_watson_machine_learning.helpers import DataConnection, S3Connection, S3Location

training_data_reference = [DataConnection(
    connection=S3Connection(
        api_key='GR9EZ1VYNCeWlvpojIEeh2R9TdFwGsrqHG3CHhLuwhIn',
        auth_endpoint='https://iam.bluemix.net/oidc/token/',
        endpoint_url='https://s3-api.us-geo.objectstorage.softlayer.net'
    ),
        location=S3Location(
        bucket='yorkugroup4-donotdelete-pr-cog5rupqyzrw31',
        path='Homicide.csv'
    )),
]
training_result_reference = DataConnection(
    connection=S3Connection(
        api_key='GR9EZ1VYNCeWlvpojIEeh2R9TdFwGsrqHG3CHhLuwhIn',
        auth_endpoint='https://iam.bluemix.net/oidc/token/',
        endpoint_url='https://s3-api.us-geo.objectstorage.softlayer.net'
    ),
    location=S3Location(
        bucket='yorkugroup4-donotdelete-pr-cog5rupqyzrw31',
        path='auto_ml/6684c698-1e14-4847-92aa-b4d8cf6d181d/wml_data/b4f804ab-b971-414f-b8df-783cb657dfbd/data/automl',
        model_location='auto_ml/6684c698-1e14-4847-92aa-b4d8cf6d181d/wml_data/b4f804ab-b971-414f-b8df-783cb657dfbd/data/automl/hpo_c_output/Pipeline5/model.pickle',
        training_status='auto_ml/6684c698-1e14-4847-92aa-b4d8cf6d181d/wml_data/b4f804ab-b971-414f-b8df-783cb657dfbd/training-status.json'
    ))

Following cell contains input parameters provided to run the AutoAI experiment in Watson Studio.

In [ ]:
experiment_metadata = dict(
   prediction_type='regression',
   prediction_column='occurrence_year',
   test_size=0.1,
   scoring='neg_root_mean_squared_error',
   project_id='e580b1d3-8955-498a-acac-f528e93283bd',
   space_id='None',
   deployment_url='https://us-south.ml.cloud.ibm.com',
   csv_separator=',',
   random_state=33,
   max_number_of_estimators=2,
   daub_include_only_estimators=None,
   training_data_reference=training_data_reference,
   training_result_reference=training_result_reference
)

<a id="inspection"></a>
# Pipeline inspection

<a id="read"></a>
## Read training data

Retrieve training dataset from AutoAI experiment as pandas DataFrame.

In [ ]:
df = training_data_reference[0].read(csv_separator=experiment_metadata['csv_separator'])
df.dropna('rows', how='any', subset=[experiment_metadata['prediction_column']], inplace=True)

<a id="split"></a>
##  Train and test data split

In [ ]:
from sklearn.model_selection import train_test_split

df.drop_duplicates(inplace=True)
X = df.drop([experiment_metadata['prediction_column']], axis=1).values
y = df[experiment_metadata['prediction_column']].values

train_X, test_X, train_y, test_y = train_test_split(X, y, test_size=experiment_metadata['test_size'],
                                                    random_state=experiment_metadata['random_state'])

<a id="preview_model_to_python_code"></a>
## Make pipeline
In the next cell, you can find the Scikit-learn definition of the selected AutoAI pipeline.

Import statements.

In [ ]:
from autoai_libs.transformers.exportable import ColumnSelector
from autoai_libs.transformers.exportable import NumpyColumnSelector
from autoai_libs.transformers.exportable import CompressStrings
from autoai_libs.transformers.exportable import NumpyReplaceMissingValues
from autoai_libs.transformers.exportable import NumpyReplaceUnknownValues
from autoai_libs.transformers.exportable import boolean2float
from autoai_libs.transformers.exportable import CatImputer
from autoai_libs.transformers.exportable import CatEncoder
import numpy as np
from autoai_libs.transformers.exportable import float32_transform
from sklearn.pipeline import make_pipeline
from autoai_libs.transformers.exportable import FloatStr2Float
from autoai_libs.transformers.exportable import NumImputer
from autoai_libs.transformers.exportable import OptStandardScaler
from sklearn.pipeline import make_union
from autoai_libs.transformers.exportable import NumpyPermuteArray
from autoai_libs.cognito.transforms.transform_utils import TAM
from sklearn.decomposition import PCA
from autoai_libs.cognito.transforms.transform_utils import FS1
from autoai_libs.cognito.transforms.transform_utils import TA1
import autoai_libs.utils.fc_methods
from sklearn.ensemble import RandomForestRegressor

#### Pre-processing & Estimator.

In [ ]:
column_selector = ColumnSelector(
    columns_indices_list=[0, 1, 3, 4, 5, 6, 7, 8, 9, 10]
)
numpy_column_selector_0 = NumpyColumnSelector(columns=[2, 3, 4, 5, 7])
compress_strings = CompressStrings(
    compress_type="hash",
    dtypes_list=["char_str", "char_str", "char_str", "char_str", "char_str"],
    missing_values_reference_list=["", "-", "?", float("nan")],
    misslist_list=[[], [], [], [], []],
)
numpy_replace_missing_values_0 = NumpyReplaceMissingValues(
    missing_values=[], filling_values=float("nan")
)
numpy_replace_unknown_values = NumpyReplaceUnknownValues(
    filling_values=float("nan"),
    filling_values_list=[
        float("nan"),
        float("nan"),
        float("nan"),
        float("nan"),
        float("nan"),
    ],
    known_values_list=[
        [
            264053604731167159695030722640794358369,
            337532398071625735104991961719159420934,
            321376099631086023744750831346280326386,
            60539795732328960037072989345448996445,
            332360496790301175934735748123422442723,
            76742912537497922071669349217007853151,
            79546493142954499173902434576116505603,
            53075182637521183331183926343883410589,
            18159194960061078393322058230133252888,
            190529973151234935076924763340533563639,
            121032545055957590314382189635113861419,
            320088263547551839428055745544558726616,
            13350588411756458079493841240047717415,
            2825508970279763986339177438394349768,
            303674803443666935351401771582589332580,
            65398268147319858581106486770706489659,
            223049754891812855143085297262772728334,
            327119034033383583968281227224468203032,
            23787671327161349531659208332642218888,
            234188068458734649347423236399039133403,
            262638634570468002689415642639704112056,
            241331659482396198436692970051660820190,
            229466453053255119038918946767010984246,
            73752048325652778457685993681742409854,
            103183831227702136496479431828710953990,
            272211577001760700170380876922554068136,
            28717371553629535107556356177504844623,
            37661473506027147727130006985383334257,
            10774622015770869574912868499502087928,
            207242260430462283337885355890841243628,
            250795583504551923973196966354843267230,
            17712880188516820075891315398112501318,
            236847904597251073653653030060353396123,
            244332269098607768414656770879494003512,
            295664761844759568007601447447815324848,
            306222826005654838389849884695830931770,
            70044919401833447958774478824606535154,
            221693029907371197027404900333511651010,
            204979701436673260388302492003333942449,
            239555411499488410164882023815372979553,
            327969616733331661264569044257371373202,
            236696617265135176985586764767613515901,
            308030645153505826737747196281553080817,
            286620622887265108815526705814327607434,
            150259708400182565208586515040592828304,
            134333772145562613400757177119221358012,
            47470567112197822112179342592195757725,
            41027847621315360869486203453712725607,
            264439182174305156014653388223207012924,
            142379381003723384859734934175053206429,
            295173193718792842465712090226478601473,
            109735563359539674180079891175030100180,
            313336803845225413438295089318202938384,
            153330143069783761206896024895328616923,
            100503805391216733876009139692801255042,
            232600927552482324283416073835509511231,
            339797397382975933019368458902277766122,
            145564045280714839477331563193019583162,
            198748570919884377381455626152258473974,
            177084609777857584653145886969218140828,
            329406106007325352216828689657743430486,
            114618238439661974642164219458924071092,
            5953117166567276399242493195860544582,
            89901766302560449282529759913697855380,
            69690405111072752393093668241881005094,
            42703759537622605839369673430029586096,
            259415281480639403390253089593652075776,
            9133474325439031453055042652347760488,
            116852317368760045424435788596753266654,
            54362222087794315774650880396231400762,
            175888303720368087492096248377555556793,
            104249495205882486899727165542084906599,
            122227337415084023348813557502636440278,
            65477642601143634483830998003108405990,
            262725573404906904859997568311178855794,
            236298615496062519667204594770477725080,
            64136263985096889290454312472295203643,
            6183079364427888697900497929949924805,
            3921541709615817869853077080862973234,
            236744672222917179004510242908651155244,
            309468618092130720482129416360661925795,
            222112027736384026134921214134345994557,
            179895535441316985627449787799843764423,
            150681349903529538726563439384969928658,
            138371694832254559843978375811161016375,
            182054846654801226859081442459890967783,
            335919100471811895344460904508047337174,
            315620757432615689222739517350297476621,
            252394924079912999809394907460002731940,
            107030010299309855234871561493633530195,
            228716060402213403702066263265795653831,
            119561808499353861361416984754188256077,
            151741207642030167139982803459773230006,
            35330799938083000716486922073928875407,
            304990340575251013605958050553449832605,
            243019884350088394886290705057530196457,
            96576736334558519253467923114865810568,
            254235962809786397743845054894413141273,
            338501244629515871754729509811606507868,
            125812825504768100862512648749703281404,
            4137321401894334792173061001611739708,
            327024964109485113569878274145191297134,
            185052351485485701287970273877095877937,
            170741310488666458805844128577342925385,
            272767060844178084547088296999309290991,
            323083200364580374965011757214665559187,
            320143878516937140708657787787599706574,
            118730029290436036171696596620532905249,
            205467091960052589395651030467871470746,
            155819539986633782189773241166738974767,
            100056704777984385095399685327508816984,
            206979657822996445066141752503156692413,
            331814287850644570040186231432417364535,
            91642651864859217748950110139153029806,
            220219287904017735661810640009794882164,
            89991568270514706781733907419643404309,
            128729629833193065910220357710765009770,
            234871432320642778512673054215912184896,
            186546611189897628203716524648664238636,
            194939894109472610376565017940518637779,
            213477527412141475986749610035181875527,
            212680364716515072896054061817249717711,
            248626060457093018774261987672357437979,
            24884916073162647497302433344891511966,
            121345580389601220634215303967870299208,
            280743461254810088942470922572648364007,
            71582570528033954527507190790104858187,
            140303469585774230905442679438034377414,
            195081289565707529545330745844543102575,
            219150228962747060773195693001268678405,
            113017070860990395598883860070552645179,
            16175905435782319654048885711494396890,
            192066956564690818438764244319198527118,
            178232143969818631606554315724689269660,
            302080238634029334438018865919607059637,
            10669106203676136486999785947371262240,
            183344263393853668972790705569234989004,
            149279006763630750476117064142802365855,
            120693756131470095102378218588192708122,
            15680368217479824346968573992730002542,
            272022696458135645512984453421781143972,
            10007292960627284852488730406146278217,
            209319093749728869983543939016071887984,
            1815578657156392504230554958133705292,
            65029245195366219111906039689492254594,
            136473211705334189802421780752770700622,
            148824573714489550872644166924896394923,
            308012379811991516132023374860638399271,
            330709733950055919114889412790677240099,
            221013855041881038570861089221153555059,
            212661407602486229191507063258784924734,
            59624511133095515570200615795860136148,
            115831832448613977923274356949019736761,
            88143374743219947883999407701151448917,
            206597897153389838633797881983144078766,
            194990480895571522864217888281118388676,
            218911371816652160837758661324560957635,
            76753593204917391764668516132853400358,
            278290096360593341298921763387073357498,
            42046017594924624555189330921831134407,
            12522284219787870588481909655714465415,
            183208679481328115563028207496078459904,
            9103895304505007716711250161822841982,
            211493611368412844221533440067248819071,
            321294166775902095535945603331648189982,
            251398708120798303159349684801702559609,
            130672145790537042234582001444765702933,
            273407603608243246543088218694139854836,
            29922265460053020304038871550394517534,
            245313280208556105506327143738401135509,
            198595570924370932214220154872148586304,
            325830353717920043321939104620005074245,
            116871063058973239524991681365361177497,
            213289739255908449662443120127236219231,
            111053086911683529002928655492830783630,
            272062870897362114056550037665373669583,
            333895036769579870672846000529334713281,
            128376934499586786668943203226712106980,
            253334861365908346193135695112637463548,
            164924609146265183906889691614759742338,
            42351392739159307283232665492486730402,
            323201456161335106730930762076510674809,
            22895275683363314015197184168757766199,
            129788247858741299231596269348245341461,
            116315138406577482797345940595263328595,
            318037041119742600910637302187952529204,
            20251186051428409483279663418886454487,
            103017571962236223992243472641056021839,
            136847139123843195632394481263308946732,
            50336103398049598512231701315675494660,
            312284717448574169720987745205389062018,
            17409764998251997100026318623522143459,
            251198596409461422482411598217785299457,
            273091656397952757848130614915831427554,
            239661201114172036679966849049768587202,
            116506374023703942873314427978774862321,
            5607118661829060509586757068326375875,
            299893000036397382037222937154829213337,
            167072474517220707581753003346483082134,
            292510177777318442212139661675910545685,
            148258213674824628163940673425765995483,
            332995333552249411962659259484833513195,
            61354732235383085707545790164175281674,
            28202098082290853436281123862833336524,
            103444592135593175234264035028447092067,
            136497762002683109686594516999938025461,
            106800921007132680229299225967028368800,
            20259433790721505258199723637986183362,
            162877041285762358919801142101952160166,
            136156346837560512200067390151619070195,
            112516422906038068192568531891745438976,
            64479146678347534216647807847333800445,
            155455608978640541749971808322476573157,
            78783057007375845984200839779198092170,
            211835294107916613855662513383165074425,
            241540619710593330370201543130531921493,
            338457576254506994640671822931821029774,
            2585954858767335188089823579163497733,
            30600077563959470062426878792641516459,
            119785752126381322581242665377131568674,
            172785089186686830424530071542586198450,
            198089606527693071364442905207999444127,
            91515211935043046247424864824209819396,
            264846789096380840223237009689429781308,
            209427636164871882790323052795246051805,
            238942328320215836691757204711434869801,
            180815872378978044874801568863111105008,
            67215280793553156641403425588759640729,
            166051587638240083764282542862052632320,
            312375134605505781337832899747919367689,
            222341102963455001216360008496713122153,
            81504300266740828429925017419260224829,
            174563499689531211972461323911076433901,
            98789522006728001300156386034312530023,
            276695983092605690186498480522758838006,
            304852019235252413349503738922053798076,
            129041349063493241175348412053992911044,
            125277698725846400589238893376476461574,
            179674567464780332166534602105330384295,
            313634704819878170432133484799656352023,
            292343422135544430419157718333254091030,
            150592851458991331417019536631300723724,
            288214383761073195061279411979461225933,
            142153524719600398239462815754242789733,
            11954165062788560897415264923542674895,
            28220698622451641538629200221037334584,
            180099254091172189083614506689853130632,
            156634293867087406940660322110826002951,
            100436206884923709010238839538079172753,
            337558991534643577500029146699110352392,
            122778188589920211585891642519799826936,
            5883175608757850438586548074190576456,
            247212747168063473314435013290371462039,
            301892036646750950291809832992841317687,
            140747889536344006527272703931518341487,
            178258585598056494544397216667057073349,
            234053123453930818984234814157697748662,
            99769791733558241108631031738410791100,
            280843966515726149081384422800767783868,
            273386998515394565262660000913297085821,
            164352170862075340104360864338204829350,
            296589027787524157532719648853712083081,
            25137609435268288815265322747087723191,
            25473423768485961841159283003069410913,
            44476833785543435121554569703632814527,
            332591382433457313917237742530091720664,
            245975212083101986651565121504126957925,
            34842572171284926161983495798584606763,
            103365375122342257852852422793726348839,
            138221754265924297424993209499077027498,
            76345777689036254563532922690612819098,
            99726367614482663251286171408826722439,
            90788218950330266227769938305090118336,
            314384291080759337532311497564121564649,
            48578108931425891888795525003259125350,
            175407263792188279273814645311619168699,
            275020443353126771469314967606019030620,
            167720898069821375914955026469738719710,
            32866652473406605330187237870025643608,
            232987252023332725841750277101540950008,
            289543022771010797765241349195228153317,
            172921772733743869078554453973990302412,
            2070198842892155554677291111431744138,
            148507748447382762146639634953776036721,
            6810501667940572110555971290487981281,
            49963364195661081428828594600690471979,
            271738905735160711203305789774014272907,
            149598629928137258954035257688141595734,
            221804355666813812596984969014593191406,
            317894609732452321469394643527151547514,
            154158977939689274384171566221559512844,
            179320019140696393616309965908014548223,
            97413859166177611905974365084684735888,
            241508252668829641246300828395099927934,
            187251065115570781749558259974842447301,
            290082931812489287087447817933597193892,
            143297426932259152978952410983222963288,
            103025785218146376626782825588271390231,
            265813893264702634801186950385970482288,
            4726938223550776026078259541021831122,
            132221096927988019353845907719206911308,
            175456158253635948937226864680780309599,
            118085189239042291764747709738031349463,
            209848205545536651344546606234377942501,
            272230625049338654397926633439912953912,
            22417130409007616128108140236378995004,
            328690026887690202599352453866385326565,
            128278620314642970168166743299025180675,
            162551644656984063714577233993547665683,
            320302316195560880576947981872347842787,
            260677691855376489910809813043793737526,
            253608556539538718654434086157197512854,
            232729416007060929956655321449573683918,
            160691354651303700942920565925669787213,
            61702967791044142830828596587410316820,
            260646698002615284973644819975245153012,
            253929455498141879143333560721793084452,
            10901297091004944383610443873093347348,
            162216410197605923196697309470218807383,
            177320856474341234127089282775680148974,
            284528807338986479128696573222660331182,
            90691226449459264953756924272957130579,
            204330140297709683191394206811949234629,
            4794507916834681295248652414992325560,
            124993276532277451769138828012537733220,
            6028549739449907445601968133501895901,
            105100821852430571338497920799415822429,
            115641136736512816524323509624574127332,
            67961879383474866737323155822109256292,
            99207145251572224526925305999274187637,
            21535065279648565296677939965847084636,
            250269356792101221824597862145107104464,
            76113896161324857498647385900381140777,
            61748145038249836677994335266954187837,
            228555672012205531920525775160155335380,
            178417740524095774679662131530221158636,
            267063327344086959690198177985025359574,
            43264672605575828717399254273055266896,
            109119538279809971367528115333407423325,
            169512829219113873578821021154586733766,
            147032073704934856313832366062608592934,
            302468677590421492195767468347091662,
            66662773950716639510175240449805406001,
            210899710498592261145111966355591999702,
            51650500339583934827042350348285529730,
            220698841172745576438834041675978640464,
            292949599172519666826665992225658261232,
            124336490876924787744670745797202609575,
            84794230782368286172857349081024891703,
            53877474892285823277695794357985523253,
            41128573055180989243166594430930297817,
            318447099461101018827689470945834602361,
            307909611813165128419027337975344813880,
            217774036405604513365275957009086945658,
            118948073173155907498148827512616574438,
            150831973150623499936092568132054000973,
            219371474420567522981529269283812915358,
            188314757005336571223392154350263797237,
            54110187314777206402527399348917367686,
            48083277711444808152570754242284126013,
            27881511311468818000281160831073190546,
            115127484967199680575098587418581293621,
            253939707757831004961152079056751921019,
            288867400334281199006263372271364870579,
            64285692916789007188547478659768803123,
            12664059146559796242024594734108998676,
            13676308175626272873625335089892639545,
            191576070833449509788878820030830367942,
            100559029006633631828529325276886942437,
            36271256232582832883011137751351076816,
            69365191587953244401611927091415440351,
            154093926036807757202604976688216942586,
            128162959948845929880886001321671794207,
            115969869789847024374704332335524507423,
            66768068861449825384527309486701834342,
            112460140717371201235196451403086286624,
            170224392863041118794387373786372781565,
            163489623313507494788369421282244009941,
            207951980500131099618264250217643515744,
            264851683896004802802415695311382023979,
            255563750893002725690463412313237602454,
            220136694132870119159535075084417082336,
            280293807174964697370586769558835693434,
            113138702879742480408062515465936839539,
            204435422581246285460383254733592546396,
            218891722056070853836573058235463868831,
            259415593277222990430935636044334628390,
            60040998729007617504803726974854874772,
            84621782703779457242060249203225729657,
            284566545507024806301094310380262873465,
            229001603445834686467187650914615522687,
            51660794588308275754731436779555835024,
            301880441736982878774371911168034782223,
            7172846723176834622595532082886506621,
            16131117566270002406826710876174530189,
            280060594980461594402204719537854554041,
            113676520631472182677609474508884385939,
            78748790591183354752126524725133682274,
            120111019159217610752605383868452079122,
            31442310286276087647448686357948519510,
            33199393047338257747581272780360104709,
            169480403731067847759050605463506945822,
            170302844302729258193035915761998207617,
            194162786694363691552200686335344489647,
            118163887045143054699517210660011756475,
            196202224110803419922889767270626128165,
            217002697711241467495903442776577075265,
            22034813516064017242300228665933817235,
            137632403859760283767316372843658929833,
            238973662937375357023640242926401991498,
            146768097509006936187082426061972326163,
            115480197942489188014590151088020189037,
            157188108414351720716375790551155237210,
            192762805643594925324915754347609285911,
            33822458337146134106856255040843499084,
            51089336644457102529449537183130985243,
            322106593940666684599705676765369148856,
            211857887727000992262927350582858251587,
            99858775131934180918485362672301530625,
            179806443066854685657849298951677106196,
            150898915964311242183544273115920190343,
            179804467391766533281426001106556013025,
            159698004156876990485770054187669256590,
            39218960128253641025999255229710839988,
            56532671550980953033909162428679400547,
            119250326923958078273991863801546806789,
            317266809206361651518772533926333184388,
            157848082185069076586048992075866481594,
            282503786907271516929045135638023175582,
            205882963216454001818554473088553259081,
            195201022272606277614473303015795879561,
            302664680001990108650514174350978737542,
            227050702858474235790341558311239885874,
            294054085807793130056068595667488165612,
            246237490600295512257359505372085673246,
            311099770410006001956397698957166187969,
            272660165634794939187059455649225084928,
            293247288697520787615352007309870792890,
            298030186831084773539614945372953695007,
            70212689773833115896842389625737365372,
            297881089415365111253448872831630223275,
            164176074145266151279728113005901769526,
            193124762281255749876134802582195355426,
            280238849663342870933431225427729115441,
            169095236245741073646728559969172737190,
            55653868187143817766935429888530095418,
            315038158620275228607102196652994305148,
            111313465703806691498813676475082411456,
            165102512329908306477805469999905771092,
            178232473429007717775173756709182512344,
            225264023243272593256331080392020235554,
            219816920429941083075700409932110871682,
            55735094744674488957771405409122924285,
            20674972040719096240123329152675264926,
            275134966196763959802276239316559560655,
            186577440173689708984282026007233978594,
            33116572161831263548967512119067121874,
            276589497888287154725185568894446067233,
            289964496762339372492451581246882338379,
            103439971104791562978788574605449898429,
            171708697204409835691490281422468442631,
            90276469162068941030953453081235515911,
            261986519623441357630490462355090465546,
            206766839592658764745325590485894371889,
            223578740793609128482781608062219136009,
            92361449110392250277482753453972890082,
            85388576424501376995328334980005160663,
            250084291869866581138751565822287877145,
            160554986765983208997740247598978001332,
            157152551545459035303091136356653297488,
            66042354822495080578871863283948338870,
            174780661692463443281156417113860573922,
            259996329110865423491828219720102191549,
            271367304564509186741671629386481325937,
            247326912141314008990546569579465136675,
            16660614252916803155783887099743288589,
            32131570778086059657126925977204878557,
            64741157690110273497882494747788354544,
            221507317939907589896050936277048087874,
            257699634065742659766399706395096648007,
            46180275334632363024165588260166428050,
            98250006563025404992700471779269262329,
            168690210574077893644780907762787198582,
            211899040709866186206734653860286266497,
            68104210876821706749464331342928764609,
            175933925159043655267056359705085691125,
            270920685427092689150751127348166076097,
            31989388796317219805548782321300588965,
            6373060424903193334462945721268506721,
            188510445484536466331029783279631101870,
            90377290754861051252539805844488708457,
            229886096679559622157658613276609867675,
            1265091083056417923165752017246722332,
            165373786996954766028976106482231500414,
            235636455618526139857810141608950657428,
            133388092830368308731490895304643906901,
            212380615095539312931159886663706250770,
            119940064653529677674038211005252372038,
            235624295186883998168347054520433148709,
            129606602173370680741536820439823099234,
            144135084137605578469876996786045609788,
            48953212076226830324193614224069133105,
            269357055323667589693219836972592867854,
            228648883048690250482459646604630077655,
            21503187622403849390609701529002151431,
            310827002060049753895839352243287722936,
            268690035643667930746985529967378277152,
            107469836006677012583642579702771072534,
            303065664116028305720772929166165886866,
            334835411885368152694078849490031124017,
            225533423746310022695478230309348854275,
            318008872698862874850688791976110643575,
            324067223131312856050072016477306453447,
            653228607786456181263169310390693493,
            120679207392792274071373811370985918127,
            197898806509026902601928347359652729540,
            27734508473397682158607917332533324499,
            31956625278623911740682840899629644987,
            132196297396817191169461457492359718590,
            261904900238741301354049129288217559412,
            251211556786069529535251162567609525750,
            264740152878722099586580164649316310058,
            185370082600327444761218730620054807451,
            157291065604187100570655042505105948835,
            19863026534754064656154004166668701418,
            242844807831790477386360099240944641121,
            4552006439274641502421804634311973134,
            238383539793382783402902967376794652417,
            213244737706152227856781714161724255001,
            230183363626854366821497366642400775396,
            261654679183687078415310527665927438528,
            184769742160075551842619249011991186619,
            94199836869310773551070788056667558331,
            258656118593502380900279635760725925787,
            65699940572586616666868842230768738916,
            208985986353859071875193265571898083716,
            281527159944859298394732599171208776450,
            144589694250249434548174960301301967399,
            147914529123805917559700900759006605584,
            240632192582107005025671156624750492336,
            45190182131930289590563600832870892360,
            44034670213512619357837601792672659199,
            50159644549874773577805077053913950763,
            26600040192328312923180500975876250342,
            53237790432089872924770815391038948669,
            253754732869204600003671515861353602660,
            26231865693660758508162308146758317676,
            123315110418797452907266068370834224918,
            231322163926059638706752107713216314176,
            257197770186117779787298123807760732425,
            191210245984584525337106032287659735534,
            173259824317834455597187722265369224491,
            166857096801449665102591235211583764517,
            169911355790558490449310064794059890290,
            102681313932764457614681841991853816041,
            57756642792807112295032963272883789564,
            54413622060220503463374670705937167059,
            199811792233140616063879888625575522313,
            94485112889266880503782437734366989949,
            209524478127331820451740163955578132641,
            218869844034714773102507996932875931423,
            96447375304032782203393041270340256791,
            221394110296058477010033827773009912125,
            158494385825910701274806104996001476409,
            126532063010083527249311426521918679301,
            146399946300307305129856558953808000156,
            36603653179869497937883501935448009481,
            97713422552152089748533408640832603543,
            188947117772659953635766450555343616310,
            82656228436557645141506927899112947912,
            329092124957838108973776550773149675615,
            55749294789099702003506565130511285650,
            250292668349845096519590297910380007358,
            48252710823496887490994027589410384437,
            208322293200603819885160873134698270658,
            244899206851617733597099345136022212564,
            93694192121816546018670667049816386793,
            38081194271602906721747084040150174824,
            38053369747977615124502898114488585810,
            34276819607644421534762518278481722514,
            15193693595761101631087156489107953812,
            229167473225988406279940993049267893566,
            332959301453066804614921948986130799541,
            285958285104547200941039594516067607194,
            137952601569323158986030351735189698220,
            88219609488656363062958959428233943945,
            335684908109651547560095371870490592444,
            43179804407455695553514095366893806513,
            241030449211084846379171413583056292341,
            316331167142046254466450560034764803384,
            176059349666716296113600723118522091636,
            91253038406145296669876037698663793698,
            258576078568346234053055199305108043069,
            138563420860217424535362621346624548487,
            334796771708050081014632933868333190528,
            192855574907662949193676091830024104706,
            5008044917254873172536409805151614476,
            201609440780259389463779050577361224923,
            312686850123569561624586897625327909606,
            183618051970998452889663092196687942200,
            292057320513523119762504668113360817102,
            275378511972578000650643472251492342449,
            320057710820521471686367284761670038305,
            53733172713702132846710745499700463606,
            129826763709435209449845921260725375954,
            24083258569358363708747906769200310929,
            8097409661138900705948068247943462294,
            19395016268865244613248871831339453508,
            264975334831324807729597927688610163317,
            231879403991176924533121609409522161956,
            129469178667394543010144274473315974408,
            274483197417493706392242858709636250351,
            67418180820544125025385973487446636547,
            336588185706724546887585296127559409066,
            124927486780232641829065765491048418157,
            218519424331261975191696924752858644920,
            45615241850722836009393888189542525962,
            295658587077012445525869136129146776806,
            15087350884030688495241864037550558036,
            195515631642323051073116037265895848690,
            256517279975652919477713049027931618132,
            150960749773601796815054438385385447545,
            135954568709658746747989376543039341708,
            217084920671418303753480314462472218478,
            236276792717192206757316563935128853891,
            241642665804929610938766105203015873902,
            11359496391293078765003092960493670832,
            60101292392584506065904736154318877105,
            289816178145127346249260771625626312805,
            294798618608970858728317210136763778542,
            287316799667085285448044583814040686915,
            128804392669044269634756267979775959951,
            230677199436094562128837274144454429182,
            314260038794014895527710042696188844797,
            339312066927474199914218712086885379647,
            298135928454004435029991786477008718193,
            261294782478740445130471318238286195166,
            57630628671578165404293037629248655285,
            81702713118296305677065649418498762406,
            83745305792795671846149641302563911790,
            336719885340898080187429980614196221623,
            327751413686729280422143663806967554467,
            56256420589526159456081063759454014463,
            241941892048825670753034804654093543019,
            216705756475992023829268141777615621301,
            25039310272814936172607544393259376588,
            148218980945990619072898116929325355314,
            327498516308454304419876120651796682388,
            304064348897265005419373654903160910672,
            117593594035869047562918444002086843999,
            277298134316657065163157741833500995201,
            319967112279094933135634368597134132671,
            128131326157396642023649350798136621839,
            15637085289377837329652157051552727151,
            151167940907172666005964227614645531620,
            3697876204555587253094843393343141927,
            93433145047231533225829373397349151068,
            337143757044842391430052796643954811361,
            235204197371583075255870563799527737971,
            163845905910295093204902749377738172927,
            222936775713475847086457549622904443396,
            255715245684635457495726227619941971753,
            283598049343125057019004071150905824245,
            13564384873607604525702472517341796192,
            318524425935655178310889745838307672594,
            111826109571488697144249257183628230437,
            334237045226246680001319762980841071431,
            72497466517507000164944883190773562926,
            90147712489209228446821001157037510653,
            144741144928640231186045623696050726342,
            225384088103198299169835986598048114257,
            243047936585936983936497416800563324776,
            328381032401956874388926361622783382518,
            50473087453962351370547216677461979132,
            89920320837744094902346380674228479369,
            326917667086005267954550097812023636638,
            133178535333166881063901922645553168917,
            170665819074043988438236726526627097105,
            290910997115419230430805584654585386735,
            327538355989590806584679400718508436484,
            292684530331201701287351134574408137025,
            321087843230444401115941513644432391341,
            42173265660221651370318441853423952279,
            41567770473043269383171740187610988549,
            115918918840730429160629677752559637998,
            299627078131605037690467609018637948897,
            45067858447907134553276210213441260941,
            51258442399850739231864367254956923740,
            305553395375031656502465580972579172995,
            142873243524339184249810696722528686640,
            218387915187123045087309521941778261449,
            76150065622092039481668091903634368701,
            75805651434199299499193478057810679507,
            166000211539036502212646825256923372681,
            325899455029000325238939312850348469364,
            305220481627219945605884700036227925951,
            132057228984575011268562122137245873100,
            155106848880741111939831117750465546236,
            325883115718711571604254351366281903880,
            62688829136705546454667192036096302028,
            42292661922511712964018194024294580805,
            301122242006829312382814813842447006138,
            71821337880780435500811063129131124538,
            172734019654447608845156547567711343490,
            232933173589717029830154703798625987652,
            188220265495567994893953207584639319881,
            307309545752505092797447155618249890189,
            75398922575982695714859578659598871001,
            43599303915315814514619170084286837978,
            273879887749132009703560074007949627480,
            192533535673119313126003946132572143098,
            212187032337529429706252977533293436785,
            84388332909080608440103777234156254267,
            327697713557879489688370335344999595037,
            317644969498005862645014607049703024951,
            190145436991477537974764677664404303179,
            289511208042320318209307597117883006761,
            75805462791499355058841988875312743319,
            272571712781484531602674785128965365895,
            230661596152055102526606422132832946645,
            55212627471055954981527032171996427168,
            277178146828656436591234543576587976360,
            198143135998347383551114094984373797473,
            279156785529481152306624673016787594973,
            27429330378720824143569341371661493904,
            22598423269641680231181096418434598644,
            66214067893162306760710715000747506012,
            45669892885844048491794163269988494453,
            271874447246392291282332677224973882846,
            20577326589263022280581894131632199426,
            162655390005767929189939602190202153674,
            29477934663103657116995528179460244426,
            197299245395093019664522747826316475526,
            270482229419378048962861958618346465183,
            58373745834956971677749816952520082196,
            177405936157163056466096620887448285551,
            37660603184782413216652418574212319119,
            46328565447602214806978637892780012144,
            51866488710127729466065014345588718880,
            286441453497700465380495556609731172967,
            126368442376186279534741881911042913366,
            133975093704359896571015265910168037706,
            218973895544172556597708816375690960789,
            205996286805187726473189286890922394204,
            284746973415759105766441925884291484949,
            50232003453664955169444527064176960004,
            36423864627995317850746468197027397346,
            216662721756367453804471248600403229015,
            291601108640189537063259874187720466734,
            157708464639680672519234937631527242343,
            239655786616082415708444303651542642642,
            133046258616937033726969539245681581382,
            96454561306187678154689730729314726028,
            56414792400828587020671859558276393635,
            87562874682659855560560086788832885176,
            4591612677046261144565493070935058472,
            63858455259562367239255782144759393216,
            34290787780325261357691844580298246001,
            12926738875508542634777788383339910258,
            150931852397350107729921187168997310505,
            309556855292912793804839855642057870873,
            193552890984457392089237201848717553513,
            226357655977495803777094719255557405682,
            208994772172981313239143620235545053222,
            221378749270605726982493742971512234804,
            313279176316370376727140276971703427829,
            266776145656351321868275376660506906124,
            146706441983559250196542959316303386628,
            324374112087837815106706127068665543080,
            16813317437667314826984935892976982725,
            208433914409203343243613571547376948050,
            201182103761974278611772669728806940239,
            150836907329493208259845107457736235555,
            53546445141808828707313751836708733082,
            223937642908469674986385533111004662881,
            4554150203223496631763061360107886749,
            243376681648094475844562028129118677428,
            334148713784795440846509641862604202267,
            84146719937642397643048720023781762207,
            42350422379686293106944037599847912710,
            194702976826978089679908840473610417367,
            156673125749790554679509331168482259798,
            238735924562841749382976340389664597197,
            302224670814491164408255343695552857790,
            333864287431040349125109750014592223766,
            11177557357900871976949636408936667630,
            159442292783337375182036617608736689314,
            221859612263749370529188545596731962938,
            301029817313103854742013261905708396620,
            280511680786637564751978285818322268621,
            218236826575376067240443468328699812919,
            260454214349526517636983063138148962391,
            290450701559856122802552468787581670756,
            17356736999506126535856698114331653927,
            154850189006458006381183848062939509082,
            81525476608997921117869697974602012172,
            297114227579335124986882083703561588863,
            209675603304455594162462794875141796314,
            272631806972752843955320210129541378843,
            16063292883361318995015576320682661634,
            251891408600276413372662383768944754057,
            335364756255220201764349842909320428714,
            213701467419926705807917451353319037184,
            360778712144514829714063954958230742,
            192672409166150049633694579420925246677,
            61598015579735477639027720835620724735,
            103873638904988831010538882950359737154,
            240969642343203168895006186482839989457,
            290432118371752304997181048311872421606,
            200854774760808506402934331891706640709,
            10235722155063823451832757795745248973,
            18860977506269639478251688492924731172,
            27564298256538952079930049220970603834,
            62958051066183364371241170868611592769,
            278350012162694503725317440973291374295,
            95739866461829842995855582100592683321,
            40444599605987456178258122062063930698,
            320772836463990681204352319082654763892,
            244980860046188122386851012869398659332,
            37068459748061510548059224515909177725,
            256392234949355145705284186868289327381,
            209375385847812539218306129837394215158,
            246444178747054364649581885511311928186,
            176219245674496375328067942515099389777,
            202458794047873630190800611591543129865,
            49814308340704382968136195946507679148,
            312004630794960118744663637882457749801,
            223446629678697801665081020786962725384,
            159019966606278102854566187774273781580,
            227583260128578556526707983023085853130,
            164669959049279926901635737044276973495,
            336601916262091968941360000786644024816,
            243635148197827188952133449842106252133,
            278028069504147403558926366686865765608,
            119931547000284473786752804930065753396,
            310013511603990436386385840721949407201,
            337796278980530814743874889438208800236,
            286434858474021259137594077663291210133,
            206431454804098676296473584872269904628,
            180077999821478707428465448304893818887,
            69152773560477709334593980419315122668,
            306143713811396421282963227125047771328,
            114563140267546433741127376081692029213,
            31348937765077283232894495513292521879,
            310176633522517587210243019226156341522,
            118323520553272626109202779867110660511,
            250812097460137782890160941645297970859,
            305124306138576402700003098360485336701,
            325165431396291965754142672594094703585,
            220845120735766246060436929620008040489,
            85090094150433318204103866020653381525,
            139565291435105998371062443048908576257,
            58059301430686513669905659265558416412,
            292420615633173301382914405676086957352,
            134211744225218262360839865486961290321,
            241646204816627157166987003864748278302,
            253513707051446319653842079400651115814,
            15299483238485916538438607943715503642,
            42071308775251710819588248335578869248,
            333174149850129294371111789265444226143,
            255714181982644575392618204368759207905,
            134692579816675751466631723731310576815,
            143272933866516020588190474073641037131,
            323784116526337722371261579347620191058,
            28166287427992896737890424972599173537,
            72546437913249191618335811080311574626,
            628041498830803007708921861908205068,
            99106600084017004831023778436334968039,
            77162130222385403566354087355065059174,
            72024364558762818441164671145690409678,
            310854826407811720422572370947365291228,
            82645340331594485500810405917371579326,
            228590426293735563405281313380838575653,
            44818356911152933948125210463423232634,
            277073442708075224239559491834361074234,
            180468632977741025569961996820591994529,
            67721289000094828158814850429974329430,
            292760232210356809010685056962282054048,
            210803415664725138030462102842623129703,
            191082065017124596889652750196817996406,
            16419820114913699891165997300121215405,
            96008484478144796934378861172639637589,
            116592761947485150870068820943308441355,
            112079269639751296980535650596158391362,
            93562223252449308218908467501192925443,
            135300067776835340559881177292657156348,
            143939256056321610338138662692129113353,
            34285774737663348489854271516308018848,
            22055726348217034417638668162555618728,
            106526362613096971343325508316871705151,
            67414768581191233849779350117205594387,
            328913976965395458376674983204740349235,
            339936103413245973834288203112289770576,
            158972237168511243944965596386778695544,
            321599244430953845397358881604499021775,
            128221860933972829182674592446112566404,
            300235249076664760880412112699799346387,
            264267888325516468490622021553223268965,
            115439058710430851496628411461383242241,
            262910852491934407355354114162846085373,
            325401721213140864839946889477893845466,
            233675423178872742352280297467038851101,
            101341860262926636480801924494460204876,
            244701334327095055262539540972560357812,
            139771649379369567345452215123735776439,
            223646661768258685870820285625867768989,
            229993390534336263469324972125258677983,
            157278657276901138404732464656864649358,
            242990272870639127475015191946565200773,
            188525889809280332309316917310434466550,
            318096223971782729748140484723577947566,
            102945516872032814520772102786897226897,
            264227175390564330249733496640445494972,
            178314150778293650532128765940741791739,
            325908239786136010281969973819080272384,
            152666074513234645310798775837058320984,
            114917019850269165504263187672817575345,
            220736573737740722912337837292306582596,
            110376636735129829696310731248096399218,
        ],
        [
            256394286445851849327317973211093951343,
            156461826602308914594492887653479514362,
            250426629706327252296893185341017863152,
            245843959895806604551258588457777142637,
            111110829565126603360440102798601025432,
            227804228969720895864432808212310003378,
            152077048157818976948997716465329263280,
            221580919537671606380080576089367119362,
            151418355741011525264639884718822727611,
            13305475334636787267136982062522695927,
            12779449812583721970903907854938843921,
            96952455064125583485348113578514691212,
            222709421771045598821009635331272353830,
            223104446718853097923293621496685355712,
            225260573473278979673968409624919302695,
            301303175123354267147503280139922820120,
            144739834394989929829131558185738310937,
        ],
        [
            131685371650839258490457910100567155591,
            286444631273086094483762359050276812325,
            171126680069511159354277377507777936943,
        ],
        [
            184134773444961152449810223251450970210,
            115599748689640764885665353205756662597,
            129887463130962091960949075068405576728,
            327743554242520425816844687950385281988,
            275186771275747256079674847794035347297,
            28941027585183773070933875733055905584,
            117002536035747381343421061787072735817,
            136304981231694750231909353516649394909,
            28626126854896900930496959175697249893,
            216850086776128870194807080835820907027,
            206997363780008914539842261918849135182,
            280368559520307680456542451636305375769,
            78270145202781458088508867296119819813,
            289905096785754060174739128875118545306,
            154931779185080215483704197044369622200,
            290432015716517013918904770628306612681,
            299051064103046497097377102540650377277,
            280373574266021586328067934989517627605,
            175147879226553475086608016245949233347,
            148459732284488970854676647752257657872,
            71429676833593956806227169714443336397,
            69942526123204340268992908901346628580,
            86805935143430695013761503142749296779,
            257066459896638158480289103367955524389,
            77471304094258662965541000437033449555,
            37204261258834609052130730899516398423,
            28958419943926887198214249476279500124,
            307565182240261845948284727961999068361,
            264853781833527382060754542433295606377,
            224614384887603092294430152906249930129,
            92075741951699127566870932474136777840,
            304052708904023183538991262644898457589,
            327101609093901108595873557198443818091,
            108239111886384298865734472643154308897,
            333395669276379213446832410868853448782,
            337224836258661191981992287373494695249,
            131109830539063619479332063848323607403,
            234644374717467666948795825823967572588,
            245845485905313516629783916155045791867,
            159165177083866620507353855021208290931,
            73287604979387863103972492620807435971,
            131745798038655201927244798896354879780,
            63252033528584515865321885249821123692,
            244518463471650099439990453130203621342,
            275086812786053645271925139973092874737,
            193867395699763002663345925358657655034,
            80955308737534951816241581274234201,
            280231050517990073911651808256075944243,
            205517225915086399321494174561364402266,
            75021565900774815185896341575813100667,
            263947886936736269552918510874917285872,
            181780163614543090197275572994796744267,
            310881248789034557772248679525149714125,
            201808265338021146108182415274592550939,
            270911707278523510824102368332603849877,
            274328149620261941533354628886312624600,
            295425594156136312878618296472455915489,
            24737427356809522571354328618421981887,
            62447994115466547679456135900021434626,
            129795354508808774170052998683241482075,
            54052840912576795187617726804936676150,
            55564550615086990624271782791227786342,
            236805611786256065839272480998886052858,
            68363571642910022966689120406195094576,
            304846700279988279389368829359611598933,
            3584040153726811900739771981225924752,
            283293049708435419612102664970510325088,
            80157647873785494471291284747801539524,
            301753720818569994604738139245452377731,
            257004625381013240820643248204765181572,
            220673394181337139181114691879080117772,
            338818909878089506587653902356832897639,
            197531067366826550605423703735681724701,
            163650688557671839873408750036387525051,
            179346778096091133202510523462128817236,
            282321190298200978879209132278537243682,
            289121776177942369369694137921923488577,
            96685514760366430075789788588272647789,
            24881061745336040170019523059581373535,
            336935016465435166905026290609626975321,
            249586014917714297423491089516575026308,
            262488759064739693628554344492620767371,
            173496854610572536284994924286535542629,
            290527646850323801003083936508651899009,
            22466453483495612109224478554005509785,
            301209648945088786655629379635168052408,
            64337905592037350427536969746242965048,
            257072138073791879454253325172400586770,
            58944449748599999507250724491312094813,
            19919211923781769233823541229312428667,
            89793158561917534375282917063123055243,
            322588112690850245538232738548568990810,
            214337747725428155806295530978892590894,
            35410089957237771018169537839560639447,
            197347856019817448140545435529018069851,
            217718111252091576744823472953477961394,
            27066347123028773582128011060964307076,
            103778235762503216269501965453973350654,
            324642466229985017511615459481020720124,
            197108892021350737834992732999007244029,
            186402670574426833921505301535977859498,
            46759751008931839569646068569051512419,
            271438371668642753002614940396417835792,
            157177788651995655897066476526072543980,
            188666652549835127722687233516524229149,
            204238142051158918125712333150426249791,
            51644333123010128013015476167857278515,
            30127411133514981625909153491925064307,
            323798183922834559342857458842850474405,
            331364745090586331789973327934619625760,
            91733967815691892298224317818786065100,
            338200336808935840052090655876312567078,
            241896934087178713628495856811599499778,
            132052858541370043297309890609660796169,
            333221447993652951814329526254216657667,
            313584716316332972556886947694486943053,
            157315861280278815236714909733883704679,
            61337157047506668707395879252768474783,
            222527639498430500144651191965366643918,
            328348854949351471967658305532145119450,
            233728041247995701142755877530761826941,
            68629637602888748000390209467280162607,
            38233958142388093395465866272962417549,
            86062130238001865624231409682158914918,
            204762332219526061406363628000394535777,
            82355434678146814682754067152159848036,
            154860795486222272830039847446278933088,
            230817127308944485859630717418768814380,
            142887622746890641974589558671141996487,
            115999336223494137186363069946269363311,
            320363130287423161388881838829064016058,
            106236787830066410388255207658425592386,
            246181413793282599559217591594350756257,
            196818016252813747492441634535591813069,
            256880580630703498120680352950335012172,
            63956342150029205781121544848574229226,
            135331022531790955342652567075433823297,
            160889180621345420797439923043731257346,
            114892219242673194941475153361224311637,
            282760949662366697164728289221173706073,
            316067712423542868732452092003927145712,
            191796019569470420298892535074708520870,
            106590524041318733348473128015875422915,
            234619861078030446586207404065756254566,
            283076892141321553292370681396493537158,
            281856192439548128804448381827935893725,
            97018119539187066288067541947675026149,
            147245359618416258613341316290829478047,
            198086951191450114928616594387126241902,
            300239582634350892152343614040994980894,
            284465320182211854367627565208946370714,
            232781943749078351228135272095694516028,
            72327029542734931264588557928229005450,
            41129587181370381062286638182456675275,
            218623391875389817468654442748116210687,
            120884165397358474331262788931037520689,
            51685138563206960153105476165146795670,
            180513835904990718452476718592571855348,
            160380306577329433951853601924083421803,
            31649605434856726418422640113870868881,
            233090410731973587192446625123542756293,
            276951417214933345005426030773586290717,
            117109778535761161679837360717423750254,
            72016518549194964864199400187305181873,
            247825185474740901903430321943388169988,
            96976093690292957642240141328635423870,
            207442494296670923136874193198517408959,
            219076820220049419088261450702421680489,
            147226474568764496555175352892763074379,
            334683225146151484603645341713247682660,
            75745879593416462784336218994670963594,
            43256540607111117317134282758838020757,
            229968858813990594545543209635313271292,
            108454537007083591901367396109039773631,
            198518135049050261392501125444849172893,
            155471858432868578286091637927086606315,
            55714103017458830181297539616443953595,
            2410297837340186731813075998373168294,
            307320856649135253633496935856453559059,
            89814544315473588118350246810592055714,
            252049891270814257306164043219588431734,
            23050819566969216176870801967340233019,
            58964077554110014699895017938266850461,
            55402373701675674648784148785337649307,
            51397443455308900388552063382663656660,
            234785956505364916291241338092988834063,
            283735896402790283455596648819761323339,
            221977952539192934525138799637905308460,
            4673362432887384955569962595583042629,
            317908126036247796034773534162551230591,
            113988807031130696721269681065987108194,
            103400973650227337788636734281677609548,
            31369701079509139512222880914567340070,
            248981898056176072768795271193565569360,
            185443961134295853393720520789851314426,
            8664400158654326525352972027934362708,
            197556410887312213275995397944862770484,
            195036246857461850461324990679357731605,
            314073502591619189644848405992002537784,
            259903188283437252725577172307739992464,
            122913006815482017661368363426986230844,
            309978274351854049154743551029871358294,
            259645453852818912354977293546317367003,
            325070569663436519866627959151886259536,
            162168723688430298826349238571666178888,
            290037656518357537595263112075453690489,
            129511612900730645282968626182780272679,
            69360106962327725929554343098183727673,
            189996396591815947868156866444841336542,
            30711984416648620260440941997175347189,
            288265302624809925868653193938063611736,
            21799760848861428709924213972196484603,
            144434013153848773440624351481209386375,
            161258659551128336216607011910320426815,
            47097150900785912822058469184092215935,
            144204917155439445686387757830713062295,
            2829009971190886819046214549868368128,
            272266545403227438814959673790957477208,
            305024109452429632906252768673403576618,
            225529392091744034261287463057459718024,
            321976774171628374426091153152095902903,
            247131932336568432620859191386816888884,
            206580617798514992060317710217218609840,
            268598240338123997427170072700502537423,
            2561693975329289021072710953846406112,
            173425810157929085007248735766293898213,
            44594893297178106406078863789852094102,
            3115417275880199020860096874417658183,
            333442346692542566186346993814469497481,
            302698270788582443340730950551972183324,
            121686096818222534863052369595863102814,
            207279646070611814618725424386392627053,
            266311175835939404724803262222939097608,
            145437090867805568108479099646070084297,
            271992521279142500809326923964411317523,
            182454206218981734388855529547596581561,
            77607598451379058399313609707531517505,
            262393038551577407636976483184421501811,
            71087574175821656063572371823608722315,
            34882434592511424475948044668065898519,
            13673805849183463455316271224282015411,
            172792175196367920968137444167786061984,
            313240797285754738164662923596548026856,
            95168657079249457368191765690805260636,
            213446613493596535212984126724143826193,
            16461399625796621492884687913351401536,
            242495621392693453993295817861023958820,
            297207195628284009524856530997919052297,
            26186149675272279246505458600736093096,
            50167584805041805455248485170834784509,
            261793719896680469624775777437758204664,
            81732237605006484453149045866346975696,
            239205197403356502367757117175836849319,
            251821024048443636916026268623417944604,
            169966560756561849309242465301787957485,
            174825587754652509387140113346003567386,
            302341859779232585779787919637489599581,
            318695420950578037904439524667775761073,
            222439734692122174192034085249372197463,
            103222430226102865796401342793759326883,
            242624783407789919956859305076542935060,
            210467248223360900860952323780918010781,
            309260299000034382183185965364845092860,
            95263564820758285939705400868855757133,
            169680195854427785131055708788574083575,
            69520925129967061741368301879351525503,
            73618574502720933507500684449889334518,
            236025638393351616053881610985292891642,
            103480117714304675489838384032328476884,
            116328633975497144202296461457904271186,
            184071751222139902632331301837194782659,
            331632123954964724581616324820979108130,
            273161046751509380082278427023215300010,
            76426095425793741611132984683134477370,
            228967526289185579511482214367952871782,
            91053579103337060210787301247436514850,
            200166693610026597193887588705006718821,
            116484689984693706544889290351459912129,
            2500877991258902453086070574717460081,
            47889229840736131778578608064586615149,
            173166523912953225594965901996572432912,
            238837728322814211801374055249425095766,
            237911202944130178946041679349087250267,
            90167629997810704707830768367509819816,
            146376703698280148597410157413136626925,
            124379776898508817334555795852862387201,
            7325641192888285428043644332235205449,
            121908833400662475302515028594967503016,
            302213113894041667963039099836125189458,
            95651550507580291641949198944435825516,
            256466904547833090450182943034177242853,
            287519876966532256800897267299749231122,
            281879534781320208872528398003925210747,
            133172544961222956777562129893518062412,
            170166929550285741898062859387541785455,
            81196119788644622049404084649648400006,
            149093680016767519396353073916981158939,
            304056959247362177551377699140922253828,
            303014462497640419729265674947143956860,
            63650235969777159778158778326766811413,
            213627911445050184757501609426722537138,
            250788113489673376139840952230046239457,
            23310855722070948617924260737920257853,
            282676316514590315177824171166528931574,
            14014323822309511591983981054986750996,
            248778733338373364266349856547533663143,
            18240123413143511852848701220336244545,
            166344972017413757875322695465975595824,
            61878869995673106039821073784849260627,
            319417345916263459111686108950956477271,
            56051093063194555147933498947352828601,
            336850460073207428495426850117477307501,
            232034430780274797922857588064155287,
            265738854794441104558516309270280658534,
            332115129331491157253657422302696209631,
            97681521355704376331967598180040265102,
            326087270025577277964016964263425390907,
            217776028723205627278975660338597418710,
            291248200731318277100919644318392465257,
            12812421742708117062534287300951972152,
            277575105730587247483499595512045254137,
            180209282883275301710000216739715032946,
            302360270884361955708819314219860331841,
            334382713023193064727693328869040616435,
            68876482991559256502686164943901841014,
            321021169898696584077187272657640046246,
            131823359317675720214132876872825741801,
            105597351033932832041904054714768674593,
            29268498391126962723084184003635117589,
            21026928272975073896360902500718273773,
            299223584370295479849269667171813267938,
            5340502123225508530021240990055483873,
            322100508379616101873130684839881536481,
            43221213741373025040429993184308651451,
            67220602499293870268539138750048249810,
            198359136634040386024110978508244995022,
            262878488216007025898033004989980519860,
            300259555037235478439128490433240870131,
            200513294892969138049099966638997403135,
            200863858360589568339593835840794301859,
            188716508227593148201023738159743752147,
            180060133591510416911185644140027953703,
            181625757271630785858107693772102158787,
            3853461959995521291877761484062343573,
            186025147966825088889953477280291686316,
            228398543778562565368287247098650911163,
            62689796697772766642219908449797165792,
            65664780855885288002325100922231098376,
            178395721975967675315363794133784783249,
            246383059736764725138972019675463442791,
            204513669452439080856233736811973829118,
            105763193502968400050738876621933472151,
            165331224609884010620023242394440488838,
            302039676529183068339991213619185910073,
            257959586971017027212993584211964493170,
            17019951457957228965742725151330578020,
            293037408410217496317704868370981332949,
            242125870067255754453540094900737933529,
            304309303750902394009566937316293804445,
            42594678982369137347393357891726511189,
            384502774606294148835805365318577166,
            96395205619409592229460686841793381634,
            91380794852337520853578526855461377381,
            56215004445606088165369494886062311659,
            210806281179005451746873561682502958803,
            246083220230207206213138035088999742518,
            233372262429087366520459599448652335389,
            180364340217633927143698938130844493753,
            233936350903861364952776560243456732608,
            247675638441598304325688407022597416269,
            134692008469215671467040435578453534521,
            43244091070163608827331221786486455130,
            282878317034783944955451513199252886738,
            284621120105317156287471674845922173672,
            21566281566872063712409816730364340791,
            128719831400953997202117169121703875696,
            193011971209158317922774488917713088458,
            145670575439844232167975489108307490667,
            176263092468088924642779691502587221417,
            13560954583790585415188516424587271719,
            40324825042825224180810175104033611533,
            19537740027017157723415280113485642278,
            144484879523627394400372913392337630022,
            186541386980406033889433621992979376748,
            272859365285434021967415766687081651610,
            337877623935642241266200586707938789755,
            100167575592839149381028169759793569418,
            79169341091123437300560075481375498262,
            75007847582958505759747658987186041161,
            234928405098737061213814097747462219911,
            64546530970428120633695625428095605166,
            234334003604442309415375505695092982638,
            320308927575082284264454288546034538756,
            288017565358343305153989400852698232544,
            61149278035211585519370346194013313672,
            210711759999311344488213327977924873649,
            266997083448003748080918899930953834611,
            92899642073007712786879604889379140575,
            219380159150936474916082160819893148296,
            293523846413731193168887866448497390451,
            330399862703349849558074592086534852453,
            126944623550968293212212438964725468858,
            116617458556710051044475172550160354100,
            185589778178864530519727238109863205254,
            141313104707836675514262564874529525126,
            59468813474128380295488082761808041881,
            30301640395613879359674444733419886358,
            94137071693688918778650277284592669655,
            29380992870531616319459006463687847337,
            184684416171614068569195126998027194583,
            181607664728909270731625150918543892217,
            76218821743827861167068631170575032777,
            74452495025650093714250159358443294795,
            1921872862555635990674915600718487981,
            178864125035611157462717230752675102593,
            215561334724073074877651421157862946623,
            310566011325324627199674244965174532532,
            131144918840671276017120381445807102349,
            84061620933782707695979894473492376747,
            315602210558110550484211339631537364036,
            263489501609522281408680462052990503885,
            252401571278286973085551955110825792504,
            269878875722925537087861515970852407463,
            36629617950377682031307661247395453301,
            50078011625999636920479659952795297669,
            104618143481722035551822983406107422129,
            67890023174781745297917141229273034443,
            197275603268282303392822906245012596879,
            148916368382742103128589356623127223071,
            50658950151375782076852169962185251118,
            335635761553788975607615910139435835339,
            139754213055157497815910262351883279085,
            237453658426425573916127790894823526945,
            316361855920790535861832493281348131181,
            129860189109567601757884450908761547455,
            96139345764430899244825395057268795243,
            71900142132596150314287958378563569055,
            249271042898555949400830767537143003027,
            148377430601192867208895748774387150513,
            108380265708615881036336677236814758019,
            119737082856350457332719133535450440454,
            197429495748347618608832019989290455574,
            149076039383105765586308313685630906809,
            81858365185591067587469468333205630847,
            84023597175626903421149995615299541285,
            249462253727801586999495245631378816999,
            317807722824332148477946565623267174470,
            7379881391610839600281980920838475165,
            3217209320313465142473856030149563734,
            321333823068816718273135270322859003088,
            267587149393442289415126819818225854058,
            170840080637689790053673307781359139772,
            279353438744312195247274747883866819663,
            23371440183782955027099891109436528456,
            95794719623456770563295267455072243157,
            85709741805528830942127300822330446747,
            183595394132781629380128008413176070716,
            139927659770353098184788473959785778067,
            270812623313571683971559767783488709085,
            12015328709513541757911116948605153361,
            268060467407309923461712575456619720701,
            100760659202970556720072689223995858649,
            331460562425037657890317087614489425440,
            279244367096500702204043332455541254611,
            2965653600967480912321743927946799679,
            121459247322414932245905197870265726482,
            315709833473787341722521231614557345437,
            176330707775475414247643650269488585432,
            169680534464045109350155275132053390976,
            266543278256338390925717264770176444685,
            135620649642137673031293660966163306784,
            140702264115695630689667647768720059900,
            258212605046262198158909530187223195798,
            58623712543942341387630314852701134219,
            73938076334009445941620500828362330356,
            269511632694321629666116796683348282978,
            129321913937582275712109871485783706437,
            318384822149498214909311825524132717809,
            122639902462693028267991986436319682861,
            249333267677145261946126594152277006375,
            88876150479795497585780405126259958667,
            80977004285324088569179739577803239275,
            149706997586868012844369010946323062586,
            39947382288988624011062227940832552399,
            109594984486461401313542721947733302038,
            240397959085609342553252628042941360024,
            272789439401435039337095708904541033958,
            88045752962866134884677886710613326,
            136313807091628040172629849391185605839,
            277452921182005797072645295846869035654,
            276305105151669345994801454666830199850,
            214820273338502303038806312419342837638,
            285176011980424964161564821563604296990,
            190912701560121951978264867022449093072,
            301320396972399107538372658642336358475,
            36499442031410774346069749151023061951,
            277701506156558306508175792398552459988,
            185069193831396220295125266808572168051,
            145741337826950957209364098820140862599,
            49750253171311837462619035603306183263,
            1890682903882927617282729309863497667,
            142478331401472416932728200681527223342,
            170981674922144326608246016748728907156,
            186384250036664765397539021337242688040,
            335416776593667717775017730400459104923,
            272651990356856239467605885445156554977,
            14093462581942618462109772977845139010,
            46464060160649643785761001795346662842,
            185735081380195980938454188608545957706,
            123005990278327948659119014432680705340,
            197690414850794203894345539945286214622,
            257797399212331088153245166690989998030,
            264710887793674218018951057496202594138,
            35558481609175753962326626741756242411,
            184912355747222126896587693900224385671,
            146601748423875814811210446844129291069,
            304250921381762161534299218553989415441,
            31451665064276944706874078524713330109,
            171318008872351541126621454298397346113,
            227792872204423522712611447428588560617,
            92828669698168894896165271357711435851,
            320286910959114291051798999709079482107,
            218046361863051112427766242199824299424,
            315352557743532525046792082519928599029,
            221701632959759921850787084750548615997,
            123781416150363212831794821353400213352,
            170596920365284279615090725143233191269,
            290663189783591488357147649616829534403,
            312688918581232019629121870755128014712,
            124296385584844075425107325137610664241,
            102835954986763758994678967238581594547,
            154345638671802830215006620208679906035,
            305284883930357866870415541262641247787,
            223854909459183871468870171074070719840,
            290106284968262710037422542221839153577,
            26502222366005978964375902458391903688,
            130035878841122767284702791007195195881,
            295289578910392598916931087210197069257,
            298794628886948189858513616111216592966,
            245798424805360566958082767731229030652,
            91205011849123162028196310563437511743,
            176189644188624096284410412641312295600,
            230990512087597791000830664878860462067,
            90822903824234014358352745427242047046,
            260951606740412572677997972198405560882,
            26905172227782122603721868881737563568,
            101870122822885958652859969287749101365,
            272078238829060817137897782500101493760,
            284617429102229953359197960470694612344,
            195631469058634136217983280132234347548,
            113473535322228134843321217819177285041,
            120576628808318369130148692114029616596,
            134876571365832562717945348672275274325,
            294622262733772132225179148333855497621,
            41022091653058257892389371918161018795,
            185858036997855069653492203524366402590,
            24632903500776308621679096424077710580,
            123313793270993802144193318723017667538,
            295884752690130045779610729577795913631,
            293580603802582921515794158570132061813,
            12618386528628826197637287328104744914,
            122633136733437683273869087042805782149,
            275613661314133341535842301074395433551,
            211638917632708994276910710135949139706,
            199183214284742353736461921667779791118,
            156393080999224410685277940744066407251,
            10805472438335520883896644822190392733,
            192886967890745693097175068614697179630,
            63538356727295951498997038420655541339,
            256034364489264781781567845244446268544,
            42716704321012080261513196294868955823,
            29428021193454422496528733870510092920,
            319748339744798461519824957129441864805,
            16450786417257942424978671892536357508,
            501222674200730189773828263174252160,
            174606837828961252291747206246770987099,
            192156504426666479855099813443367379789,
            212556417176664878276238922286169588730,
            13881318297597377532491456778263880509,
            193802835305360662510674809358914778815,
            217079655422141160017775519335245289457,
            178516246514146215194926398586949989964,
            83858233133032359336683005009128337851,
            216919758985617582695709825657039607875,
            330592415508443643239777804094619368788,
            90501152453752025854857728423475499412,
            315998027615148128644113674639351929267,
            81720413362364591698546416980425242574,
            262395744511018416247353630549008188133,
            308930908279946180193545693128891577699,
            72722955371738130313363502251523620909,
            169156551039641879434747542492568167303,
            315370032034749703091691216569282311922,
            78932796164639592571410216730862577766,
            88376522876474210474499426675895125043,
            100050607104361253820583045407734113409,
            196529312219172191961873021660083674691,
            303276465807956231391346522383862445375,
            16235157798144984679541535303012235418,
            335704937174187776866970137213863453298,
            101502405150294714052506373157876661391,
            116293939540983044925923984744127249009,
            17504334248245368176090890644611105610,
            237508677801471726388883764780045888388,
            126022952700979430111771438982258160892,
            202191268238514664272651184653177668842,
            308914589078587698479372016004607396225,
            323837757577586003526032586945272953081,
            292645662337677307457495303882916195571,
            104772797243571034048953129553960410691,
            328145189895765192068890801172818580368,
            316201444973817090078156032966887664902,
            11352480148026530882386167511203623679,
            126930112630418141958320610506515205260,
            84374207098729629294175236249277923927,
            151308826569091517205497411412950168237,
            175515781106883416006719240220029345934,
            275593171161021701181737629040030518195,
            68047412314947960022304468694347491476,
            160023091488252059190183769391505282625,
            250385196088295694178222630263808652169,
            266026039204050209694573534490410040569,
            184710784698440654636334610399832209540,
            85935263128190069654586375960697827497,
            6266487157517068646440455107010021010,
            65882065989874604523624903089581026996,
            34614402613705664559787470053363758538,
            229552915843639881426624656836504750747,
            38504996922156008074430815006313603475,
            90003023831411671909635278215489844884,
            134150857255982728582253663921011985835,
            37397397307709056167780298806325076905,
            257513392373035239925622092251970804142,
            331868094145121518681448640799646738238,
            213197676451438196370264648841393704181,
            321512776173218901614288876964917671184,
            273004615962927816155569497591871705320,
            111780716028919177645133400451748313115,
            37158116873309877540037165434386110391,
            296489859724637694240677824021144778084,
            258963725389380193345173193831041048610,
            93386114720610299577816140333538858262,
            315085222920197642818726248924142750798,
            244323843009479745947109237585848636512,
            112001285796269894783119470235345467016,
            32386561822794873790640621686362955531,
            18963842429594547142884852181239003765,
            214796603959895409563308215090843447388,
            17969734765656846048970400035718340129,
            68743921076740100512654844861553572453,
            19110115192417311626859971578855029079,
            33881988607873121851974029986366439929,
            100472675184567526038999817034772938930,
            85794078473302716887849786272679809694,
            274746628626761991149481935218582587635,
            323905129249114421151635436020295648245,
            8223334316520195152001564141785221845,
            183836232274087841662899815317968793183,
            47146351426226053260551888810323561412,
            338223559147671041127172187599289668638,
            175576643008142232330921548405059280459,
            321360111051547477638469741787416009268,
            34088335614028917345267737176423483868,
            142771823164836742689979101683698099393,
            316464674401153285487102095841771727930,
            328005043687108531471272677230774100775,
            277487719636137992891029683757102491170,
            251335923078300141220042787984291385665,
            10529820198163383667154135274716020498,
            156342170458633565162030086325267876043,
            204389129372628812622295349631872292318,
            64444903299355629592567379085209977311,
            288869225359738208192960334998393885754,
            13557896100497205996242403803449202307,
            268341489370426443147059641900170895196,
            306257842855590230359493237419981521989,
            116334461928341274623121967827958410246,
            226150758976189466046768921953695243909,
            196537002670807870938906559750711640865,
            203149134269817604580786224757474854865,
            272195131052939249315926362823310789119,
            114303359618989238403003349875502055220,
            326951061859031893977464234167428617393,
            332795175761133282577131879807023138735,
            176997864597265208091630457026925985959,
            118133441390324570126801370426921070722,
            97146796453880274874485966448315314561,
            119274424986574436771517401084020410560,
            40685444361097604430782039049822754941,
            166665505004739697685113796724225908618,
            68271398264256279195357636143540056563,
            63123988016595608915013715093588964699,
            3057051919719808806916102534580426428,
            126363867600529666646014267204834312435,
            226874730686445140640350937053617114141,
            187486959879721215575355942461496739829,
            273199754038245431277875434555495742876,
            31403507098163215231369743537383987796,
            210281743019866278087785215212533220162,
            50031740975102047134687531091417610979,
            321271324922449934996940884903427798456,
            327298481993893433215456523726562977358,
            318219518582767619514606970225374645857,
            241808183904421594845716007252873572299,
            222406314577422810923324621059658783130,
            289895002968627706302812556647784503847,
            285802477219779331835158268266194122503,
            314285283585827521687714401646507598642,
            122163037220122402460118911723904179518,
            196010510456956746305125158876361749452,
            92519485656184573025986882702899582580,
            27978183751341055803613272564532591853,
            109053274453922495967862555685909006439,
            64646007853731812306850050206513116711,
            64379142709519198796727188200587130116,
            337690959331312891041157855022148541874,
            129267679123211062268266205130187655286,
            308806308365312535102787992704170558885,
            68122267385320406384671729270204567716,
            114342092229602330408121907469638795665,
            8607793527342709239547005947844021476,
            31202207458416438718384497622045683914,
            329852978866271284663055482621537817617,
            108079228180194763601489244112778514302,
            87491311086579133150640275130467160038,
            129815010345195288140363025434323383241,
            321580835348865096724207855542388820844,
            233993937460041200286497731035223973636,
            254572531587014495446231162542095345793,
            117155355233208191192362712136500098747,
            102443357737147151442538240142764927999,
            247982708456126749332292272365713624265,
            223178149863033337583548849943700168939,
            57190127154067634704265218640262302502,
            71457344361316927255613835513246744189,
            270400480054479685744253029440291521021,
            22776538348668971132049147724381011165,
            36071831312412234115395843743556433027,
            333379064259036819047113229015021461959,
            238881180324274202590991549549963731236,
            312584987876558464389282074579289076559,
            196973819291131064426212581403729538468,
            47030505744698137502722931508848704074,
            146398884901177900629264617439133351988,
            296620028342673884619261079085449498339,
            66074879847304824146053105877796988270,
            49325420661045342723942631490276820448,
            71346121715776656224220792023566020867,
            54864573379949039020235890438477571075,
            161336603958490273882958993914466596722,
            88944566928731778356401142476720512542,
            289073976010397271838355046549435579651,
            33611435630132909516953551875963587390,
            337246392888442238133921785781963162404,
            67230707655057226195610837982595083130,
            178058587930477781896761277708369048600,
            322158565528775414302624922128455854339,
            245973380719170216240764044620817814225,
            317560711783192844923775220958642190141,
            2254769300294779952604190439629120587,
            222422539520073386368451838877412404192,
            11409210670802373297891632915631583493,
            197087910734195768833534159770011903164,
            300062251661827452417257724806197998608,
            55632171069066165206561081635494961164,
            115424449124443232280942431123935232189,
            145777291717787203461584102967945905116,
            76169562995947298316715933528739843043,
            36673213936530590313299842112577963870,
            83018657590237788760613480111821028806,
            192018714791361603890978019114004929153,
            277717525134726691485171036060501798540,
            20029909418908665380862989553892132818,
            227369193261402268494281252040539245000,
            135291162172345550479405410115612175123,
            101688130644681124407727564429861293652,
            213203018828620042868524130418041483596,
            123646118362334397359042647992496826483,
            53240228281758006005709738028480314905,
            101655627176625412739522329532246765698,
            302685689149507261553137013924235438212,
            293390469348530572934210556723786469932,
            207914410996930721958964398931418858396,
            46837791312406371982033290416927341858,
            243241777633165074568187185863068575710,
            231856149043706998924828539481205729486,
            16729215794335827009707324910647778523,
            267903335641994576070557240131790700719,
            30982377942373179057541357031139182938,
            142200929537301343474321461310296784726,
            315001601423529633711450894749191086628,
            9220557413583603081065710656722770666,
            326665522520549220662390567310799858689,
            114614502058307916437487363695611101149,
            18888462107996279234903822776899164417,
            118731754850954571865871362168701549292,
            55835332980513064951459277471792899361,
            283520530174648233516242865309620883399,
            306571330521524321053793656499460020239,
            188748206497670755267454587595943811717,
            281222475509230339421233306991244254413,
            287515274971446817317097982778561373017,
            267683686541285909225444995181487172185,
            188502441343089354349064717950611179983,
            135751032175335355403356120942494264140,
            292908953652237722321788131992325641284,
            188093868379848411407074884672280356644,
            286741270022765836075652000235789848492,
            240153790657133333689425313146874080873,
            67802270468985238047081543845231019814,
            13292390677534438346917601443201410404,
            78652845244050278334223090016861925411,
            10394693379016406493438433232097951415,
            238343666791295117916164150635850913076,
            21023345140328664205388002037515101432,
            166441158071638918852644128100898581048,
            110239113942584255635713669413580233906,
            50907337588551557547161071457919261879,
            316855009641993700124906248440217504959,
            79493195953937833756009738409984777160,
            306737056319534439781632926941431515454,
            279626790129322077984695146020573245340,
            331724588396390881785449634944623414583,
            320199211109247243228648778943467602188,
            320115852643595812794446743384252577898,
            128835612046896036587970628240172546128,
            274609260955705011607273197196747474654,
            9585831943661581640018891118583853873,
            148328643915523221399622985577011098061,
            285981013598118789775509032559982336831,
            72560760317226385404285282881329363649,
            314482662675884150981779468888733940704,
            229740931574110678102991819203236402546,
            148630415531495842571347203172563182324,
            293756826107937057380255308407058350106,
            326829153618741685038804608594224734209,
            12810386694467806514945280003651019451,
            31556838079410484039260787279900662779,
            62287513724939325475471776083832385678,
            28559277987003724604199408845604676646,
            321432035569017434399813790221032228029,
            181829109113165840653989100575667148068,
            250657975900800670792563263957974205957,
            114980614540058708272622901028273857030,
            81839735119564219857996266737480432778,
            76308149921594315485518960636184892399,
            257978900978301247549646155905751981032,
            239670478815762455299709175206183665503,
            209844309908020641870219286901270869657,
            247150013587046981440141349498176223142,
            81081634752831487577428724966725130774,
        ],
        [
            294464545141722516097256280731894975474,
            124094839933410723683878912956295011019,
            224310167635422833447226369205437526781,
            28640338323513895230029169349131856259,
            122748704260997320388799634521755261699,
            294560352401394478523310715485740477898,
            256779801474077123805915349544442285660,
            103962408816972492706093752315054202036,
            125643757622261086702077936210187403767,
            84533613778061418702711492342543524474,
            188111785105307315661278329513128793986,
            71015519890949125944474427601412527528,
            259179572326455633571135113392603456391,
            136561995901486449994099164932738766268,
            337711361179447443292779782615722737022,
            125900424945478884886523713473333179422,
            334512469410722719972429441562686473670,
            9434063446218448847895254075418589559,
            79145931217068868627331715195267685995,
            131599531781181906479285578775122787802,
            57738334810763669216657698782307736524,
            253394753538039562573128740950911436653,
            191287412829131931859352595270244973101,
            27882324048785842565208712539459275764,
            283587262870573946249367097058137747336,
            113452599557274008519172545206950106901,
            294142733022580636405052291224240239722,
            165836296691736010131258499654330313613,
            242314934718653079063719037991622481540,
            182918071525955543777196401051660778048,
            290511890424828630408226206834935183777,
            60941063647000600329372352257127942925,
            198635869631995652062182668242926733311,
            142384780111387485455467753769476423706,
            230792540350371443438074715635214197565,
            163994099600826245788487895020050351079,
            326809981321703736016349131665920062560,
            314947998934400955113072995398119178558,
            162511252020922643551971984626132054421,
            71997111432205161196217749581249021317,
            36556142232996703588358914567900258860,
            217683200506843043995304647540837746164,
            111301158505535464341536937935919422898,
            20094460610428286285115152210551055439,
            244010445491035391892736641519175808630,
            199298087795073836174437620538469661320,
            69130940878405557326440674705371450912,
            326344170959085889599893349494187075134,
            1483877584200414885904485694421977019,
            97664821211514220224480786056583275456,
            48680600032596927433421082065136232819,
            131456349852864736509460665294254582327,
            23893768973337809605964445276023931510,
            239085023648164931989934763667374973532,
            124284432429808884264581721737228782706,
            165555779897742937728387132779884551251,
            339321137694453399151003628668474699112,
            153494584261995274744637991855330683855,
            44751306620297457134909480576555552463,
            116627390100535368184077733795238743118,
            169191911697623569135106397584425650842,
            22770250578311054271975696617449515064,
            256066262962295627357616007006898257463,
            181441872202365506704797697010455346222,
            277093107033377156857862271953323789702,
            252204104962232912861923277319718581644,
            325277754187105847795790080734610973059,
            303844059940969948087580185793446587801,
            30430323577554513590368840852660452158,
            38072685336560901124341792538101470536,
            209237683745927712321526300662938025845,
            191818365691465929111780703875931512624,
            5245728865895278856511189020622520385,
            66571255953228800244033381543445851180,
            43616257843727029457393815290409893462,
            119922260406723194191294916779227991886,
            149290913887794476942131680033511968517,
            188442022941292348777740731462162029526,
            94339823885425841415480416669082875609,
            47330062137038346008401221299999319342,
            145564402725520721366206424054489164080,
            208296328887028935770855562047048181972,
            175160700109822510192522873160692420998,
            214175428798666401739431151723426683546,
            25201599174485061121573211614843274300,
            48618751323027860591023872942602710574,
            264689931847687473667244699686697943519,
            322649780690749475489651946199501581182,
            229128723415156048634261914614350899975,
            21845773242550664903026508113247335979,
            304226432920801275484309899048304826163,
            193178617466040411502857395099346816269,
            28803907585959837686930191745251000028,
            42668682517896339995794139771037136844,
            96460266034657464200650657072441202089,
            214562309801357571149865914884756497793,
            25475138284346891398600892519969837461,
            178133163408037147067617054399016416496,
            58324525702081471645363386865239212909,
            10676768855010769494942584121791287661,
            265692791910244131818755772475046280527,
            52257029525527950117885930490824824353,
            232438962145050625166751828113894780830,
            161687099069410992812913095689740232660,
            164315909666362368039550849109310713400,
            295859592172134484757362463545328844417,
            128481097364021114199363714505738005947,
            194519783928106182211577987967505516559,
            17351191489239648225846550995520971874,
            178823325405694355346691118161054553911,
            310991058454187786937532383647977277400,
            104810079066670404208517695039164941061,
            36895138240422675864402918184806113263,
            125130797208204723826709173069560346062,
            189162472476036022883672929007669717712,
            90671221888246589038800119477431048007,
            236839896033569814385783108564771274548,
            104582114184921375396905215769275009902,
            332771074894324573792956619361532407102,
            260342772939684110335486499997262370005,
            210763571021190194841665405464306158207,
            76492096414594860411382059948973213299,
            113012134862730592888648940878603863023,
            243697435663474468407118408964127886104,
            6336320306347990741344322373497369304,
            323945166910320650200415896451849568403,
            48220075919363629846342152615210151757,
            90245618586267090558180572364017495395,
            199947838250806598712699172351873790909,
            298758298095402820252284101966685106907,
            143545991039424188945521972815610655848,
            58647929690859150177381083079215449129,
            302604268909362811846228189230886309582,
            12232935165523690756263916914619130274,
        ],
    ],
    missing_values_reference_list=["", "-", "?", float("nan")],
)
cat_imputer = CatImputer(
    strategy="most_frequent",
    missing_values=float("nan"),
    sklearn_version_family="23",
)
cat_encoder = CatEncoder(
    encoding="ordinal",
    categories="auto",
    dtype=np.float64,
    handle_unknown="error",
    sklearn_version_family="23",
)
pipeline_0 = make_pipeline(
    numpy_column_selector_0,
    compress_strings,
    numpy_replace_missing_values_0,
    numpy_replace_unknown_values,
    boolean2float(),
    cat_imputer,
    cat_encoder,
    float32_transform(),
)
numpy_column_selector_1 = NumpyColumnSelector(columns=[0, 1, 6, 8, 9])
float_str2_float = FloatStr2Float(
    dtypes_list=[
        "float_num",
        "float_num",
        "int_num",
        "float_num",
        "float_num",
    ],
    missing_values_reference_list=[],
)
numpy_replace_missing_values_1 = NumpyReplaceMissingValues(
    missing_values=[], filling_values=float("nan")
)
num_imputer = NumImputer(strategy="median", missing_values=float("nan"))
opt_standard_scaler = OptStandardScaler(
    num_scaler_copy=None,
    num_scaler_with_mean=None,
    num_scaler_with_std=None,
    use_scaler_flag=False,
)
pipeline_1 = make_pipeline(
    numpy_column_selector_1,
    float_str2_float,
    numpy_replace_missing_values_1,
    num_imputer,
    opt_standard_scaler,
    float32_transform(),
)
union = make_union(pipeline_0, pipeline_1)
numpy_permute_array = NumpyPermuteArray(
    axis=0, permutation_indices=[2, 3, 4, 5, 7, 0, 1, 6, 8, 9]
)
tam = TAM(
    tans_class=PCA(),
    name="pca",
    col_names=[
        "X",
        "Y",
        "event_unique_id",
        "Division",
        "homicide_type",
        "occurrence_date",
        "Hood_ID",
        "Neighbourhood",
        "Lat",
        "Long",
    ],
    col_dtypes=[
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
    ],
)
fs1_0 = FS1(
    cols_ids_must_keep=range(0, 10),
    additional_col_count_to_keep=12,
    ptype="regression",
)
ta1 = TA1(
    fun=np.square,
    name="square",
    datatypes=["numeric"],
    feat_constraints=[autoai_libs.utils.fc_methods.is_not_categorical],
    col_names=[
        "X",
        "Y",
        "event_unique_id",
        "Division",
        "homicide_type",
        "occurrence_date",
        "Hood_ID",
        "Neighbourhood",
        "Lat",
        "Long",
        "pca_0",
        "pca_1",
        "pca_2",
        "pca_3",
        "pca_4",
        "pca_5",
        "pca_6",
        "pca_7",
        "pca_8",
        "pca_9",
    ],
    col_dtypes=[
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
        np.dtype("float32"),
    ],
)
fs1_1 = FS1(
    cols_ids_must_keep=range(0, 10),
    additional_col_count_to_keep=12,
    ptype="regression",
)
random_forest_regressor = RandomForestRegressor(
    max_depth=5,
    max_features=0.9999999993788795,
    n_estimators=30,
    n_jobs=4,
    random_state=33,
)


Pipeline.

In [ ]:
pipeline = make_pipeline(
    column_selector,
    union,
    numpy_permute_array,
    tam,
    fs1_0,
    ta1,
    fs1_1,
    random_forest_regressor,
)

<a id="train"></a>
## Train pipeline model


### Define scorer from the optimization metric
This cell constructs the cell scorer based on the experiment metadata.

In [ ]:
from sklearn.metrics import get_scorer

scorer = get_scorer(experiment_metadata['scoring'])

<a id="test_model"></a>
### Fit pipeline model
In this cell, the pipeline is fitted.

In [ ]:
pipeline.fit(train_X,train_y)

<a id="test_model"></a>
## Test pipeline model

Score the fitted pipeline with the generated scorer using the holdout dataset.

In [ ]:
score = scorer(pipeline, test_X, test_y)
print(score)

<a id="next_steps"></a>
# Next steps

#### [Model deployment as webservice](https://github.com/IBM/watson-machine-learning-samples/tree/master/cloud/notebooks/python_sdk/deployments/autoai)
#### [Run AutoAI experiment with python SDK](https://github.com/IBM/watson-machine-learning-samples/tree/master/cloud/notebooks/python_sdk/experiments/autoai)  

<a id="copyrights"></a>
### Copyrights

Licensed Materials - Copyright © 2020 IBM. This notebook and its source code are released under the terms of the ILAN License.
Use, duplication disclosure restricted by GSA ADP Schedule Contract with IBM Corp.

**Note:** The auto-generated notebooks are subject to the International License Agreement for Non-Warranted Programs  
(or equivalent) and License Information document for Watson Studio Auto-generated Notebook (License Terms),  
such agreements located in the link below. Specifically, the Source Components and Sample Materials clause  
included in the License Information document for Watson Studio Auto-generated Notebook applies to the auto-generated notebooks.  

By downloading, copying, accessing, or otherwise using the materials, you agree to the <a href="http://www14.software.ibm.com/cgi-bin/weblap/lap.pl?li_formnum=L-AMCU-BHU2B7&title=IBM%20Watson%20Studio%20Auto-generated%20Notebook%20V2.1">License Terms</a>  

___